In [2]:
import pandas as pd
import numpy as np


In [13]:
# === Load dataset ===
print("📥 Loading full dataset...")
df = pd.read_csv("C:/Users/sana_/Desktop/Master/sem2/advance network sec/archive (4)/csv/data.csv")


📥 Loading full dataset...


In [48]:
df.shape

(8656767, 86)

In [14]:
print("Total samples:", len(df))
# === Show unique label distribution before any processing ===
print(df['Label'].value_counts())
print("\nUnique classes:", df['Label'].unique())

Total samples: 8656767
Label
Malicious    8655466
Benign          1301
Name: count, dtype: int64

Unique classes: ['Benign' 'Malicious']


In [15]:
# Step 1: Filter only benign rows
benign_df2023 = df[df['Label'] == 'Benign'].copy()


In [16]:
print("Total samples:", len(benign_df2023))
# === Show unique label distribution before any processing ===
print(benign_df2023['Label'].value_counts())
print("\nUnique classes:", benign_df2023['Label'].unique())

Total samples: 1301
Label
Benign    1301
Name: count, dtype: int64

Unique classes: ['Benign']


In [17]:
# Step 2: Save to CSV (in the same folder as your notebook/script)
benign_df2023.to_csv('benign_df2023.csv', index=False)


In [18]:
# === Load dataset ===
print("📥 Loading full dataset...")
benign = pd.read_csv("C:/Users/sana_/Desktop/Master/sem2/advance network sec/archive (4)/benign_df2023.csv")


📥 Loading full dataset...


In [ ]:
benign.info()

In [25]:
benign.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Len

In [36]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# === Load original binary benign dataset ===

n_original = len(benign)
n_target = 5000
n_synthetic = n_target - n_original

# === Initialize new DataFrame ===
synthetic_df = pd.DataFrame()

# === Helper Functions ===
def generate_private_ip():
    ranges = [('10.0.0.0', 8), ('172.16.0.0', 12), ('192.168.0.0', 16)]
    base_ip, subnet = random.choice(ranges)
    base_parts = list(map(int, base_ip.split('.')))
    max_hosts = 2**(32 - subnet) - 2
    host_id = random.randint(1, max_hosts)
    ip_int = (base_parts[0]<<24) + (base_parts[1]<<16) + (base_parts[2]<<8) + base_parts[3] + host_id
    return '.'.join(str((ip_int >> (i*8)) & 0xFF) for i in reversed(range(4)))

def assign_ports(protocol):
    if protocol == 6:  # TCP
        return random.randint(1024, 65535), random.choice([80, 443, 22])
    elif protocol == 17:  # UDP
        return random.randint(1024, 65535), random.choice([53, 123])
    return random.randint(1024, 65535), random.randint(1024, 65535)

# === Generate fields ===
synthetic_df['Protocol'] = np.random.choice([6.0, 17.0], size=n_synthetic)
synthetic_df['Src IP'] = [generate_private_ip() for _ in range(n_synthetic)]
synthetic_df['Dst IP'] = [generate_private_ip() for _ in range(n_synthetic)]
synthetic_df['Src Port'], synthetic_df['Dst Port'] = zip(*[assign_ports(p) for p in synthetic_df['Protocol']])
synthetic_df['Flow ID'] = synthetic_df['Src IP'] + '-' + synthetic_df['Dst IP'] + '-' + \
                          synthetic_df['Src Port'].astype(str) + '-' + \
                          synthetic_df['Dst Port'].astype(str) + '-' + \
                          synthetic_df['Protocol'].astype(str)

start_time = datetime(2023, 1, 1, 0, 0, 0)
synthetic_df['Timestamp'] = [start_time + timedelta(seconds=i*random.randint(1, 3)) for i in range(n_synthetic)]

# === Always Benign ===
synthetic_df['Label'] = 'Benign'

# === Sample traffic types from real values ===
synthetic_df['Traffic Type'] = np.random.choice(benign['Traffic Type'].dropna().unique(), size=n_synthetic)
synthetic_df['Traffic Subtype'] = np.random.choice(benign['Traffic Subtype'].dropna().unique(), size=n_synthetic)

# === Generate realistic values for all numeric columns ===
numeric_cols = benign.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    mean, std = benign[col].mean(), benign[col].std()
    synthetic_df[col] = np.random.normal(loc=mean, scale=std * 0.8, size=n_synthetic)

# === Reorder and combine ===
final_df = pd.concat([benign, synthetic_df], ignore_index=True)
final_df = final_df[benign.columns]  # exact order




In [37]:
print("Total samples:", len(final_df))
# === Show unique label distribution before any processing ===
print(final_df['Label'].value_counts())
print("\nUnique classes:", final_df['Label'].unique())

Total samples: 5000
Label
Benign    5000
Name: count, dtype: int64

Unique classes: ['Benign']


In [38]:
# === Save result ===
final_df.to_csv("2023augmented_benign_5000.csv", index=False)
print("✅ Final dataset created with only benign samples: 5000 rows total.")

✅ Final dataset created with only benign samples: 5000 rows total.


In [40]:
# === Load dataset ===
print("📥 Loading full dataset...")
B = pd.read_csv("C:/Users/sana_/Desktop/Master/sem2/advance network sec/archive (4)/2023augmented_benign_5000.csv")
M= pd.read_csv("C:/Users/sana_/Desktop/Master/sem2/advance network sec/archive (4)/sampled_malicious.csv")



📥 Loading full dataset...


In [42]:
print("Total samples:", len(M))
# === Show unique label distribution before any processing ===
print(M['Label'].value_counts())
print("\nUnique classes:", M['Label'].unique())

Total samples: 5000
Label
Malicious    5000
Name: count, dtype: int64

Unique classes: ['Malicious']


In [43]:
# Combine
balanced_df2 = pd.concat([B, M], ignore_index=True)

In [45]:
print("Total samples:", len(balanced_df2))
# === Show unique label distribution before any processing ===
print(balanced_df2['Label'].value_counts())
print("\nUnique classes:", balanced_df2['Label'].unique())

Total samples: 10000
Label
Benign       5000
Malicious    5000
Name: count, dtype: int64

Unique classes: ['Benign' 'Malicious']


In [47]:
# === Save result ===
balanced_df2.to_csv("TENNNNNNN100002023TENK.csv", index=False)
